**BAG OF VISUAL WORDS**

In [1]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold   
import keras
from keras.datasets import cifar10
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics

import os
from scipy import ndimage
from scipy.spatial import distance

In [2]:
!pip install opencv-contrib-python==4.4.0.44

Loading Dataset

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Merging train and test data

In [4]:
x = np.concatenate((x_train,x_test),axis = 0)
y = np.concatenate((y_train,y_test),axis = 0)
print('X shape: ', x.shape)
print('Y shape: ', y.shape)

X shape:  (60000, 32, 32, 3)
Y shape:  (60000, 1)


Sift Operation

In [5]:
def SIFT(image):
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(image,None)
    return kp,des

def sift_features(images,labels_g):
    sift_vectors = {}
    descriptor_list = []
    count = 0
    labels = []
    for image in images:
        features = []
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        kp, des = SIFT(image)
          
        if des is not None:
            descriptor_list.extend(des)
            features.append(des)
            sift_vectors[count] = features        
            labels.append((count,int(labels_g[count])))
        count += 1
    return [descriptor_list,sift_vectors,labels]

sifts = sift_features(x,y) 
descriptor_list = sifts[0]
bovw_feature = sifts[1] 
sift_labels = sifts[2]

KMeans

In [6]:
k = 100
def kmeans(k, descriptor_list):
    kmeans = KMeans(n_clusters = k, n_init=1)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_ 
    return visual_words
    
# Taking the central points which will be visual words    
visual_words = kmeans(k, descriptor_list) 

Creating Histogram of k clusters and no. of images having those clusters

In [7]:
imageFeatures = np.zeros((len(sift_labels), k), "float32")
for i in range(len(sift_labels)):
    words, distance = vq(bovw_feature[sift_labels[i][0]][0],visual_words)
    for word in words:
      imageFeatures[i][word] += 1

In [8]:
std = StandardScaler().fit(imageFeatures)
imageFeatures = std.transform(imageFeatures)

In [9]:
len(imageFeatures)

59880

In [10]:
sift_labls = []
for i in range(len(sift_labels)):
  sift_labls.append(sift_labels[i][1])
X_train, X_test, y_train, y_test = train_test_split(imageFeatures, sift_labls, test_size=0.1667, random_state=4)

KNN model training

In [11]:
# Training the data using knn
clf = cv2.ml.KNearest_create()
clf.train(X_train, cv2.ml.ROW_SAMPLE, np.asarray(y_train, dtype=np.float32))

True

In [12]:
ret, results, neighbours ,dist = clf.findNearest(X_test, k=10)

Accuracy

In [13]:
predicted_label = []
for var in results:
    label = var
    predicted_label.append(int(label))

print (y_test)
print (predicted_label)
    
# Measuring the accuracies
metrics.accuracy_score(y_test, predicted_label)

[9, 7, 0, 5, 4, 5, 5, 0, 5, 1, 0, 7, 2, 3, 8, 0, 5, 6, 9, 8, 3, 5, 1, 8, 9, 9, 2, 4, 2, 9, 4, 1, 4, 0, 0, 7, 8, 6, 9, 7, 6, 7, 3, 1, 3, 4, 2, 3, 0, 4, 0, 1, 5, 5, 7, 2, 5, 5, 9, 6, 9, 3, 4, 9, 1, 9, 6, 0, 5, 2, 8, 8, 5, 5, 0, 8, 4, 0, 6, 3, 6, 9, 8, 0, 2, 9, 9, 5, 6, 4, 6, 3, 2, 3, 2, 6, 5, 6, 6, 9, 0, 5, 0, 0, 6, 1, 9, 0, 4, 5, 9, 0, 7, 9, 2, 2, 1, 3, 1, 1, 6, 0, 2, 2, 6, 0, 9, 3, 0, 5, 8, 1, 7, 0, 9, 6, 0, 9, 3, 0, 8, 6, 1, 0, 5, 0, 8, 4, 4, 3, 2, 5, 2, 0, 0, 9, 1, 2, 0, 9, 2, 7, 1, 4, 4, 1, 1, 3, 9, 0, 6, 6, 5, 5, 0, 2, 0, 1, 0, 7, 5, 4, 7, 9, 2, 7, 3, 2, 7, 9, 5, 7, 0, 7, 4, 9, 6, 1, 8, 4, 1, 5, 3, 1, 5, 7, 6, 3, 6, 4, 6, 3, 0, 3, 3, 1, 1, 8, 4, 5, 7, 5, 9, 3, 9, 7, 1, 0, 6, 8, 8, 9, 7, 7, 4, 6, 8, 0, 4, 5, 1, 9, 4, 3, 1, 5, 4, 1, 8, 7, 7, 8, 6, 5, 1, 2, 2, 7, 0, 6, 9, 8, 5, 8, 7, 1, 8, 4, 5, 2, 6, 9, 5, 3, 7, 2, 9, 8, 0, 6, 5, 0, 6, 7, 9, 1, 5, 3, 4, 3, 2, 9, 6, 7, 2, 8, 8, 0, 3, 2, 0, 3, 9, 3, 3, 2, 2, 5, 6, 8, 5, 6, 5, 2, 0, 3, 1, 4, 0, 9, 4, 8, 0, 4, 9, 6, 9, 4, 2, 7, 9, 7, 8, 

0.1640953716690042

In [15]:
c = 0
for i in range(len(y_test)):
  if y_test[i] == predicted_label[i]:
    c+=1
print(len(y_test))
print("No. of matched images in test data")
print(c)

9982
No. of matched images in test data
1638
